In [ ]:
import numpy as np
from transformers import BertTokenizer, TFBertForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

sentiment_mapping = {
  0: "Very Negative",
  1: "Negative",
  2: "Neutral",
  3: "Positive",
  4: "Very Positive"
}

In [ ]:
text = "I like my job"

tokens = tokenizer(text)

predictions = model.predict([tokens.input_ids])

predicted_class = np.argmax(predictions.logits)

sentiment_mapping[predicted_class]

1/1 [==============================] - 0s 114ms/step


'Positive'

In [ ]:
text = "I hate my job"

tokens = tokenizer(text)

predictions = model.predict([tokens.input_ids])

predicted_class = np.argmax(predictions.logits)

sentiment_mapping[predicted_class]

1/1 [==============================] - 0s 238ms/step


'Very Negative'

#### Analyze Setiments in Dataset

In [ ]:
import pandas as pd

csv_file = "../IMDB.csv"
dataframe = pd.read_csv(csv_file)

dataframe

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
rows_to_analyze = dataframe.sample(5)

sentiments_dataframe = rows_to_analyze.copy()

sentiments_bert = []

rows_to_analyze

,review,sentiment
46622,Perhaps I'm out of date or just don't know wha...,negative
35104,"Ride with the Devil, like Ang Lee's later Brok...",positive
4645,Martin Ritt's first film offers an exceptional...,positive
16937,Hercules the Avenger is by far the best single...,positive
33081,Words can scarcely describe this movie. Loaded...,negative


In [ ]:
for review in rows_to_analyze['review']:
  tokens = tokenizer(review)

  predictions = model.predict([tokens.input_ids])

  predicted_class = np.argmax(predictions.logits)

  sentiments_bert.append(sentiment_mapping[predicted_class])

sentiments_dataframe['sentiments_bert'] = sentiments_bert

sentiments_dataframe

1/1 [==============================] - 0s 485ms/step


,review,sentiment,sentiments_bert
46622,Perhaps I'm out of date or just don't know wha...,negative,Negative
35104,"Ride with the Devil, like Ang Lee's later Brok...",positive,Very Positive
4645,Martin Ritt's first film offers an exceptional...,positive,Positive
16937,Hercules the Avenger is by far the best single...,positive,Positive
33081,Words can scarcely describe this movie. Loaded...,negative,Very Negative


#### Analyze Setiments in Arabic Dataset

In [2]:
from camel_tools.sentiment import SentimentAnalyzer

model = SentimentAnalyzer("CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")

In [4]:
tsv_file = "../ar_reviews.tsv"
dataframe = pd.read_csv(tsv_file, sep='\t')

label_column = dataframe.pop("label")
dataframe.insert(1, label_column.name, label_column)

dataframe

,text,label
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,Positive
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,Positive
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,Positive
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,Positive
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,Positive
...,...,...
99994,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...,Negative
99995,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...,Negative
99996,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...,Negative
99997,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...,Negative


In [9]:
rows_to_analyze = dataframe.sample(5)

sentiments_dataframe = rows_to_analyze.copy()

sentiments_bert = []

rows_to_analyze

,text,label
30108,وجدت نفسي أثناء القراءة أتسائل نفس أسئلة لكن ا...,Positive
72166,تحبي تقولي ايه يافندم في ريفيو الكتاب ؟؟؟ اقول...,Negative
70845,تجربه سيئه فى مكان عظيم . . الخدمه سيئه للغايه...,Negative
90796,القصه متكرره جدا ، والنهايه متوقعه تماما ، يوج...,Negative
12879,يستحق التجربة . الاستقلالية في الجناح لغرفتين ...,Positive


In [10]:
for review in rows_to_analyze['text']:

  predictions = model.predict([review])

  sentiments_bert.append(predictions[0])

sentiments_dataframe['sentiments_bert'] = sentiments_bert

sentiments_dataframe

,text,label,sentiments_bert
30108,وجدت نفسي أثناء القراءة أتسائل نفس أسئلة لكن ا...,Positive,positive
72166,تحبي تقولي ايه يافندم في ريفيو الكتاب ؟؟؟ اقول...,Negative,negative
70845,تجربه سيئه فى مكان عظيم . . الخدمه سيئه للغايه...,Negative,negative
90796,القصه متكرره جدا ، والنهايه متوقعه تماما ، يوج...,Negative,negative
12879,يستحق التجربة . الاستقلالية في الجناح لغرفتين ...,Positive,positive
